In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

exec(open("crud.py").read())





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "admin"
password = "test"
shelter = AnimalShelter()


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open('logo.png', 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Nicholos Tyler 4/17/21'))),
    html.Hr(),
    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                          html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                          html.Button(id='submit-button-two', n_clicks=1, children='Mountain Rescue'),
                          html.Button(id='submit-button-three', n_clicks=2, children='Disaster Rescue'),
                          html.Button(id='submit-button-four', n_clicks=3, children='Reset'),
                  ]),
    #html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),  
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
    Input('submit-button-three', 'n_clicks'),Input('submit-button-four', 'n_clicks'),
              ])

def on_click(n_clicks, value, b3, b4):
    #start case
    if (int(n_clicks) == 0):
        #breed_list = ["Labroador Retriever Mix", "Chesapeake Bay Retriever", "NewFoundland"]
        df = pd.DataFrame.from_records(shelter.read(
            {
                "breed": "Labrador Retriever Mix",
                "sex_upon_outcome": "Intact Female",
                #"age_upon_outcome_in_weeks": {"$range": [26,156]}
            }))
    #use higher number of button clicks to determine filter type
    elif (int(value) == 1):
        df = pd.DataFrame.from_records(shelter.read(
            {
                "breed": {"$in": [{"breed":"Labroador Retriever Mix"}, {"breed":"Chesapeake Bay Retriever"}]},
                "sex_upon_outcome": "Intact Female",
                #"age_upon_outcome_in_weeks": {"$range": [26,156]}
            }))
    elif (int(b3) == 2):
        #print('water')
        df = pd.DataFrame(list(shelter.read({"animal_type":"Other"})))
    elif (int(b4) == 3):
        #print('water')
        df = pd.DataFrame(list(shelter.read({})))
        
    return df.to_dict('records')


@app.callback(Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    return [
        dcc.Graph(            
            figure = 131
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
    
def update_map(viewData):
        dff = pd.DataFrame.from_dict(viewData)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.75,-97.48], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[1,9])
                    ])
                ])
            ])
        ]
app